In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import torch

In [ ]:
dataset = pd.read_csv("heart.csv")
dataset.head()

In [ ]:
dataset['ChestPainType'].unique(), dataset['RestingECG'].unique(), dataset['ExerciseAngina'].unique(), dataset['ST_Slope'].unique()

In [ ]:
dataset = pd.get_dummies(dataset, columns=['Sex', 'ChestPainType', 'RestingECG', 'ST_Slope'])

In [ ]:
dataset = pd.concat([pd.Series(np.where(dataset['ExerciseAngina'].to_numpy() == 'Y', 1, 0), dataset.index).rename('Angina'), dataset], axis=1).drop('ExerciseAngina', axis=1)

In [ ]:
train_dataset = dataset.drop('HeartDisease', axis=1)
y = dataset['HeartDisease']

In [ ]:
scaler = MinMaxScaler()
X = pd.DataFrame(scaler.fit_transform(train_dataset), columns=train_dataset.columns)

In [ ]:
X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.1)
X_train.shape, X_validate.shape

In [ ]:
train_features = torch.tensor(X_train.to_numpy())
train_labels = torch.tensor(y_train.to_numpy())

validation_features = torch.tensor(X_validate.to_numpy())
validation_labels = torch.tensor(y_validate.to_numpy())

In [ ]:
n_features = train_features.shape[1]
model = torch.nn.Sequential(torch.nn.Linear(n_features, 50),
                                            torch.nn.ReLU(),
                                            torch.nn.Linear(50, 1),
                                            torch.nn.Sigmoid())

In [ ]:
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.01)

In [ ]:
n_batches = 2
train_features_batched = train_features.reshape(n_batches,
                                               int(train_features.shape[0]/n_batches),
                                               train_features.shape[1])
train_labels_batched = train_labels.reshape(n_batches,
                                            int(train_labels.shape[0]/n_batches))

In [ ]:
n_epochs = 2000
loss_list = []
validate_loss_list = []
 
for epoch in range(n_epochs):
    for batch_idx in range(n_batches):
        optimizer.zero_grad()
         
        outputs = model(train_features_batched[batch_idx].float())
         
     
        loss = criterion(outputs.flatten().float(),
                         train_labels_batched[batch_idx].float())
     
         
        loss.backward()
         
        optimizer.step()
         
    outputs = model(train_features.float())
     
    validation_outputs = model(validation_features.float())
     
         
    loss = criterion(outputs.flatten().float(),
                     train_labels.float())
     
    validate_loss = criterion(validation_outputs.flatten().float(),
                              validation_labels.float())
     
    loss_list.append(loss.item())
     
    validate_loss_list.append(validate_loss.item())
 
print('Finished Training')

In [ ]:
import matplotlib.pyplot as plt
plt.plot(loss_list, linewidth=3)
plt.plot(validate_loss_list, linewidth=3)
plt.legend(("Training Loss", "Validation Loss"))
plt.xlabel("Epoch")
plt.ylabel("BCE Loss")
